In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ld65slcs
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ld65slcs
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=967e606aa3687d1962d698c14bc0f06343785bb0297ef089da88c09fb2bfcb01
  Stored in directory: /tmp/pip-ephem-wheel-cache-oh1mhnw_/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
%matplotlib inline
import keras
from keras.optimizers import Adam
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers import LeakyReLU, Dropout, Activation, Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose
import cv2
import re
import datetime
import skimage
import scipy # 
import keras_contrib
from glob import glob
from IPython.display import Image
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from random import random

from keras.initializers import RandomNormal


tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [0]:
class DataLoader():
    def __init__(self, img_res=(256, 256)):
        self.img_res = img_res
        _URL = 'https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz'

        path_to_zip = tf.keras.utils.get_file('facades.tar.gz',origin=_URL,extract=True)

        self.PATH = os.path.join(os.path.dirname(path_to_zip), 'facades/')

    
    def load_data(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path=os.path.join(self.PATH,'train') if not is_testing else os.path.join(self.PATH,'test')
        train_path = glob('%s/*.jpg' %(path))
#         label_path = glob('%s/*.png' %(data_path))
        batch_images = np.random.choice(train_path, size=batch_size)


        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = img_A = cv2.imread(img_path).astype(np.float)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]
#             h, w, _ = img.shape
#             _w = int(w/2)
#             img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = cv2.resize(img_A, self.img_res)
            img_B = cv2.resize(img_B, self.img_res)

            # If training => do random flip
            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        data_path=os.path.join(self.PATH,'train')
        train_path = glob('%s/*.jpg' %(data_path))

        self.n_batches = int(len(train_path) / batch_size)

        for i in range(self.n_batches-1):
            batch = train_path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = img_A = cv2.imread(img).astype(np.float)

                h, w, _ = img.shape
                _w = int(w/2)
                img_A, img_B = img[:, :_w, :], img[:, _w:, :]

                img_A = cv2.resize(img_A, self.img_res)
                img_B = cv2.resize(img_B, self.img_res)
                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

In [0]:
class cygan():
    def __init__(self, lambda_val):
        #input shape
        self.img_rows = 256
        self.channels = 3
        self.img_shape = (self.img_rows,self.img_rows,self.channels)
        self.resnet_block_num = 9
        self.batch_size=1
        self.losses = [[]]
        #output of PatchGAN
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)
        
        #data loader
        self.loader = DataLoader()

        #lossweights
        self.lambda_cycleloss = lambda_val
        self.lamba_idloss = 0.1*self.lambda_cycleloss
        
        #optimizer
        opti = Adam(0.0002,0.5)
        
        #building and compiling the discriminators
        self.d_1 = self.discriminator()
        self.d_1.compile(loss = 'mse',optimizer = opti,metrics = ['accuracy'])
        self.d_2 = self.discriminator()
        self.d_2.compile(loss = 'mse',optimizer = opti,metrics = ['accuracy'])
        #building the generators
        self.g_1 = self.generator()
        self.g_2 = self.generator()
        #input image
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)
        
        fake_B = self.g_1(img_A)
        cycle_A = self.g_2(fake_B)
        fake_A = self.g_2(img_B)
        cycle_B = self.g_1(fake_A)
        self.d_1.trainable = False
        self.d_2.trainable = False
        
        valid_A = self.d_2(fake_A)
        valid_B = self.d_1(fake_B)
        
        self.combined = Model([img_A,img_B],[valid_A,valid_B,cycle_A,cycle_B])
        self.combined.compile(loss=['mse','mse','mae','mae'],loss_weights=[1,1,self.lambda_cycleloss,self.lambda_cycleloss],optimizer = opti)
    
    #defining the generator with three components - encoder, resnet block, decoder
    def generator(self):
        
        def resnet_block(kernel, input_layer):
            init = RandomNormal(stddev=0.02)
            model = Conv2D(kernel, kernel_size=(3,3),padding='same',kernel_initializer=init)(input_layer)
            model = InstanceNormalization(axis=-1)(model)
            model = Activation('relu')(model)
            model = Conv2D(kernel, kernel_size=(3,3),padding='same',kernel_initializer=init)(model)
            model = InstanceNormalization(axis=-1)(model)
            model = Concatenate()([model,input_layer])
            return model    

        
        #c7s1-64
        init = RandomNormal(stddev=0.02)
        input_layer = Input(shape = self.img_shape)
        model = Conv2D(64, kernel_size=(7,7),strides=(1,1),padding='same',kernel_initializer=init)(input_layer)
        model = InstanceNormalization(axis=-1)(model)
        model = Activation('relu')(model)
        #d128
        model = Conv2D(128, kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=init)(model)
        model = InstanceNormalization(axis=-1)(model)
        model = Activation('relu')(model)
        #d256
        model = Conv2D(256, kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=init)(model)
        model = InstanceNormalization(axis=-1)(model)
        model = Activation('relu')(model)
        #R256
        for resnet in range(self.resnet_block_num):
            model = resnet_block(256,model)
        #u128
        model = Conv2DTranspose(128, kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=init)(model)
        model = InstanceNormalization(axis=-1)(model)
        model = Activation('relu')(model)
        #u64
        model = Conv2DTranspose(64, kernel_size=(3,3),strides=(2,2),padding='same',kernel_initializer=init)(model)
        model = InstanceNormalization(axis=-1)(model)
        model = Activation('relu')(model)
        #c7s1-3
        model = Conv2D(3, kernel_size=(7,7),strides=(1,1),padding='same',kernel_initializer=init)(model)
        model = InstanceNormalization(axis=-1)(model)
        out_img = Activation('relu')(model)

        return Model(input_layer,out_img)
    
    #discriminator is Patch GAN architecture same as Pix2Pix
    def discriminator(self):
        init = RandomNormal(mean=0.0, stddev=0.02, seed=123)

        def single_layer(input_img, filters, filter_size = 4, batch_norm = True, kern_init = init):
            """layer in the discriminator"""
            model = Conv2D(filters, filter_size,strides = 2,padding ='same',kernel_initializer=kern_init)(input_img)
            if batch_norm:
                model = InstanceNormalization(axis=-1)(model)
            model = LeakyReLU(alpha=0.2)(model)
            return model

        i = Input(shape = self.img_shape)

    #         Concatenating image and label image


        l1 = single_layer(i,64,batch_norm = False)
        l2 = single_layer(l1,128,)
        l3 = single_layer(l2,256)
        l4 = single_layer(l3,512)

        l5 = Conv2D(1,(4,4),strides = 1,padding = 'same',activation = 'sigmoid')(l4)
        return Model(i,l5)

    
    def train(self,epochs = 100,batch_size = 1, sample_interval = 50):
        
        start_time = datetime.datetime.now()
        valid=np.ones((batch_size,)+self.disc_patch)
        fake=np.zeros((batch_size,)+self.disc_patch)

        for epoch in range(epochs):
    #         start=time.time()
            for batch_i, (imgA,imgB) in enumerate(self.loader.load_batch(batch_size)):
                fakeA = self.g_2.predict(imgB)
                fakeB = self.g_1.predict(imgA)
                
                d_1_real_loss = self.d_1.train_on_batch(imgB,valid)
                d_1_fake_loss = self.d_1.train_on_batch(fakeB,fake)
                d_1_loss = 0.5 *np.add(d_1_real_loss,d_1_fake_loss)
                
                d_2_real_loss = self.d_2.train_on_batch(imgA,valid)
                d_2_fake_loss = self.d_2.train_on_batch(fakeA,fake)
                d_2_loss = 0.5 *np.add(d_2_real_loss,d_2_fake_loss)
                
                d_total_loss = 0.5*np.add(d_1_loss,d_2_loss)
                
                g_loss = self.combined.train_on_batch([imgA, imgB], [valid,valid, imgA,imgB])
                elapsed_time = datetime.datetime.now()-start_time
                print("Epoch - %d, Batch - %d, D_loss %f acc %3d%%, (G_loss %f, adv_loss %f, cycle_loss %f) , time %s" %(epoch,batch_i, d_total_loss[0], d_total_loss[1]*100, g_loss[0],np.mean(g_loss[1:3]),np.mean(g_loss[3:5]),elapsed_time))
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                if batch_i == self.loader.n_batches-1:
                    self.losses.append((d_total_loss,g_loss))
                
#         save_models(epoch)
        
    def sample_images(self, epoch, batch_i):
        os.makedirs('cycleganimages/', exist_ok=True)
        r, c = 2, 3

        imgs_A, imgs_B = self.loader.load_data(batch_size=1, is_testing=True)
        fake_A = self.g_2.predict(imgs_B)
        fake_B = self.g_1.predict(imgs_A)
        cycle_A = self.g_2.predict(fake_B)
        cycle_B = self.g_1.predict(fake_A)
        gen_imgs = np.concatenate([imgs_B, fake_A,cycle_B, imgs_A,fake_B,cycle_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("cycleganimages/lambda_%d_%d_%d.png" % (self.lambda_cycleloss,epoch, batch_i))
        plt.close()
        
#     def save_models(self,epoch):
#         filename1 = 'g_model_AtoB_%06d.h5' % (epoch+1)
#         self.g_1.save(filename1)
#         # save the second generator model
#         filename2 = 'g_model_BtoA_%06d.h5' % (epoch+1)
#         self.g_2.save(filename2)
#         print('>Saved: %s and %s' % (filename1, filename2))
    
    def show_losses(self):
        losses = np.array(self.losses)

        fig, ax = plt.subplots()
        plt.plot(losses[0], label='Discriminator')
        plt.plot(losses[1], label='Generator')
        plt.title("Training Losses")
        plt.legend()
        fig.savefig("cycleganimages/lambda_%d_Loss.png" %(self.lambda_cycleloss))
        
        plt.show()

In [0]:
cycleGAN = cygan(10)



In [0]:
cycleGAN.train(epochs = 20)

Epoch - 9, Batch - 85, D_loss 0.000001 acc 100%, (G_loss 9.153187, adv_loss 0.998663, cycle_loss 0.357793) , time 1:14:02.048228
Epoch - 9, Batch - 86, D_loss 0.000000 acc 100%, (G_loss 11.761122, adv_loss 0.998782, cycle_loss 0.488178) , time 1:14:03.241573
Epoch - 9, Batch - 87, D_loss 0.000001 acc 100%, (G_loss 9.011677, adv_loss 0.998674, cycle_loss 0.350716) , time 1:14:04.416233
Epoch - 9, Batch - 88, D_loss 0.000001 acc 100%, (G_loss 8.749843, adv_loss 0.998660, cycle_loss 0.337626) , time 1:14:05.610720
Epoch - 9, Batch - 89, D_loss 0.000001 acc 100%, (G_loss 10.542496, adv_loss 0.998555, cycle_loss 0.427269) , time 1:14:06.782676
Epoch - 9, Batch - 90, D_loss 0.000001 acc 100%, (G_loss 8.974468, adv_loss 0.998707, cycle_loss 0.348853) , time 1:14:07.978893
Epoch - 9, Batch - 91, D_loss 0.000001 acc 100%, (G_loss 9.542013, adv_loss 0.998461, cycle_loss 0.377255) , time 1:14:09.162091
Epoch - 9, Batch - 92, D_loss 0.000001 acc 100%, (G_loss 11.700485, adv_loss 0.998633, cycle_lo

In [0]:
cycleGAN1 = cygan(5)
cycleGAN1.train(epochs = 20)

30171136/30168306 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch - 0, Batch - 0, D_loss 0.339980 acc  40%, (G_loss 9.367560, adv_loss 0.357778, cycle_loss 0.865200) , time 0:02:16.472606


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch - 0, Batch - 1, D_loss 0.313731 acc  51%, (G_loss 8.081037, adv_loss 0.450031, cycle_loss 0.718097) , time 0:04:04.110098
Epoch - 0, Batch - 2, D_loss 0.290096 acc  60%, (G_loss 7.891266, adv_loss 0.547147, cycle_loss 0.679697) , time 0:05:30.314356
Epoch - 0, Batch - 3, D_loss 0.268774 acc  63%, (G_loss 7.544127, adv_loss 0.516394, cycle_loss 0.651134) , time 0:06:56.934067
Epoch - 0, Batch - 4, D_loss 0.271707 acc  65%, (G_loss 6.438654, adv_loss 0.543671, cycle_loss 0.535131) , time 0:08:22.839527
Epoch - 0, Batch - 5, D_loss 0.245146 acc  67%, (G_loss 6.596380, adv_loss 0.580499, cycle_loss 0.543538) , time 0:09:48.352925
Epoch - 0, Batch - 6, D_loss 0.249146 acc  68%, (G_loss 6.135725, adv_loss 0.525263, cycle_loss 0.508520) , time 0:11:14.113446
Epoch - 0, Batch - 7, D_loss 0.233806 acc  70%, (G_loss 7.470408, adv_loss 0.576382, cycle_loss 0.631764) , time 0:12:40.349846
Epoch - 0, Batch - 8, D_loss 0.244009 acc  69%, (G_loss 6.515527, adv_loss 0.590653, cycle_loss 0.533422

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch - 0, Batch - 51, D_loss 0.105093 acc  89%, (G_loss 6.386505, adv_loss 0.707376, cycle_loss 0.497175) , time 1:15:47.730100


In [0]:
cycleGAN2 = cygan(15)
cycleGAN2.train(epochs = 20)

In [0]:
cycleGAN3 = cygan(1)
cycleGAN3.train(epochs = 20)

In [0]:
from google.colab import files
!zip -r cycleganimages.zip /content/cycleganimages
files.download('cycleganimages.zip')

In [0]:
cycleGAN.losses